<a href="https://colab.research.google.com/github/josuecaldas/ManualML_IV_Aplicaciones/blob/main/PYTHON_predict_income.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### SET UP DATA

In [3]:
# Librerias de tratamiento
# ========================

import pandas as pd
import numpy as np
from google.colab import files

In [81]:
# Librerias de preprocesamiento y modelado
# ========================================

from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import r2_score

In [ ]:
#data = pd.read_stata("predict_income_2020.dta")
#data = pd.read_stata("predict_income_2020.dta", convert_categoricals=True, preserve_dtypes=True)

In [60]:
data = pd.read_csv("predict_income_2020.csv", encoding = "latin-1")

In [72]:
data

,Unnamed: 0,p207_20,p208a_20,p104_20,p110a1_20,p1121_20,p1123_20,p1124_20,p1125_20,p1126_20,...,p401f_20_2,p401f_20_3,p558c_20_2,p558c_20_3,p558c_20_4,p558c_20_5,p558c_20_6,p558c_20_7,p558c_20_8,p558c_20_9
1,2,2.0,56.0,3.0,1.0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,3,1.0,25.0,3.0,1.0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,4,1.0,24.0,3.0,1.0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5,2.0,52.0,4.0,1.0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,6,2.0,54.0,3.0,1.0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12072,12073,2.0,32.0,3.0,1.0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
12073,12074,2.0,27.0,3.0,1.0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
12094,12095,1.0,57.0,3.0,1.0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12096,12097,1.0,30.0,3.0,1.0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
data = data.dropna()

In [73]:
# Division de las variables predecidas y predictores
# ==================================================

predecido = ['ingmo2hd']
predictores = list(set(list(data.columns)) - set(predecido))

In [74]:
# División de las variables en "x" y "y"
# ======================================

y = data[predecido]
x = data[predictores]

In [75]:
# Division en test set y training set (test set = 30%)
# ====================================================

x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size = 0.30,
                                                    shuffle = True, 
                                                    random_state = 22)
print(x_train.shape); print(x_test.shape)

(5558, 144)
(2383, 144)


#### REGRESION LASSO

In [103]:
# Seleccionamos el mejor alpha (lambda)
# =====================================

lasso_alphas = np.logspace(-10, 2, 300)
lasso_r = LassoCV(alphas = lasso_alphas)

In [ ]:
# Entrenamos el modelo
# =====================================

lasso_r = lasso_r.fit(x_train, y_train)

In [78]:
# Obtenemos los coeficientes LASSO
# =================================================

lasso_coeficientes = pd.DataFrame(
                      {'predictor': x.columns,
                      'coef':lasso_r.coef_.flatten()}
                  )

#lasso_coeficientes

In [112]:
# Obtenemos el MSE y el coeficiente de determinación en el training set
# =====================================================================================

pred_train_lasso= lasso_r.predict(x_train)
print(f"El MSE en el training set es {np.sqrt(mean_squared_error(y_train, pred_train_lasso))}")
print(f"El coeficiente de determinación en el training set es {(r2_score(y_train, pred_train_lasso))}")

El MSE en el training set es 20290.524471419652
El coeficiente de determinación en el training set es 0.4641682256381494


In [113]:
# Obtenemos el MSE y el coeficiente de determinación en el test set
# ==================================================================

pred_test_lasso= lasso_r.predict(x_test)
print(f"El MSE en el test set es {np.sqrt(mean_squared_error(y_test, pred_test_lasso))}")
print(f"El coeficiente de determinación en el test set es {(r2_score(y_test, pred_test_lasso))}")

El MSE en el test set es 18515.680310862514
El coeficiente de determinación en el test set es 0.5025063630339148
